In [0]:
%sh 
sudo wget https://downloads.apache.org/kafka/3.5.1/kafka_2.12-3.5.1.tgz

--2023-10-03 21:57:01--  https://downloads.apache.org/kafka/3.5.1/kafka_2.12-3.5.1.tgz
Resolving downloads.apache.org (downloads.apache.org)... 88.99.95.219, 135.181.214.104, 2a01:4f9:3a:2c57::2, ...
Connecting to downloads.apache.org (downloads.apache.org)|88.99.95.219|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 106956505 (102M) [application/x-gzip]
Saving to: ‘kafka_2.12-3.5.1.tgz.1’

     0K .......... .......... .......... .......... ..........  0%  139K 12m32s
    50K .......... .......... .......... .......... ..........  0%  278K 9m24s
   100K .......... .......... .......... .......... ..........  0%  278K 8m20s
   150K .......... .......... .......... .......... ..........  0% 1.15M 6m37s
   200K .......... .......... .......... .......... ..........  0%  157K 7m30s
   250K .......... .......... .......... .......... ..........  0% 59.9M 6m15s
   300K .......... .......... .......... .......... ..........  0% 80.1M 5m22s
   350K .......... .......

In [0]:
%sh 
tar -xvf kafka_2.12-3.5.1.tgz

kafka_2.12-3.5.1/
kafka_2.12-3.5.1/LICENSE
kafka_2.12-3.5.1/NOTICE
kafka_2.12-3.5.1/bin/
kafka_2.12-3.5.1/bin/connect-distributed.sh
kafka_2.12-3.5.1/bin/connect-mirror-maker.sh
kafka_2.12-3.5.1/bin/connect-standalone.sh
kafka_2.12-3.5.1/bin/kafka-acls.sh
kafka_2.12-3.5.1/bin/kafka-broker-api-versions.sh
kafka_2.12-3.5.1/bin/kafka-cluster.sh
kafka_2.12-3.5.1/bin/kafka-configs.sh
kafka_2.12-3.5.1/bin/kafka-console-consumer.sh
kafka_2.12-3.5.1/bin/kafka-console-producer.sh
kafka_2.12-3.5.1/bin/kafka-consumer-groups.sh
kafka_2.12-3.5.1/bin/kafka-consumer-perf-test.sh
kafka_2.12-3.5.1/bin/kafka-delegation-tokens.sh
kafka_2.12-3.5.1/bin/kafka-delete-records.sh
kafka_2.12-3.5.1/bin/kafka-dump-log.sh
kafka_2.12-3.5.1/bin/kafka-e2e-latency.sh
kafka_2.12-3.5.1/bin/kafka-features.sh
kafka_2.12-3.5.1/bin/kafka-get-offsets.sh
kafka_2.12-3.5.1/bin/kafka-jmx.sh
kafka_2.12-3.5.1/bin/kafka-leader-election.sh
kafka_2.12-3.5.1/bin/kafka-log-dirs.sh
kafka_2.12-3.5.1/bin/kafka-metadata-quorum.sh
kafka_2.1

In [0]:
mkdir arquivos

In [0]:
ls

arquivos/   hadoop_accessed_config.lst*  logs/
azure/      kafka_2.12-3.5.1/            metastore_db/
conf/       kafka_2.12-3.5.1.tgz         preload_class.lst*
eventlogs/  kafka_2.12-3.5.1.tgz.1


In [0]:
%sh
./kafka_2.12-3.5.1/bin/kafka-topics.sh --bootstrap-server localhost:9092 --create --topic genomic --partitions 1 --replication-factor 1

SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/databricks/jars/----ws_3_3--mvn--hadoop3--org.apache.logging.log4j--log4j-slf4j-impl--org.apache.logging.log4j__log4j-slf4j-impl__2.18.0.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/databricks/driver/kafka_2.12-3.5.1/libs/slf4j-reload4j-1.7.36.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.apache.logging.slf4j.Log4jLoggerFactory]
Created topic genomic.


In [0]:
%sh
./kafka_2.12-3.5.1/bin/kafka-topics.sh --bootstrap-server localhost:9092 --list

SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/databricks/jars/----ws_3_3--mvn--hadoop3--org.apache.logging.log4j--log4j-slf4j-impl--org.apache.logging.log4j__log4j-slf4j-impl__2.18.0.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/databricks/driver/kafka_2.12-3.5.1/libs/slf4j-reload4j-1.7.36.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.apache.logging.slf4j.Log4jLoggerFactory]
genomic


In [0]:
!pip install kafka-python

     |████████████████████████████████| 246 kB 6.3 MB/s 
You should consider upgrading via the '/local_disk0/.ephemeral_nfs/envs/pythonEnv-f909cf92-4dea-4e19-9c2e-6f02ed22e496/bin/python -m pip install --upgrade pip' command.


In [0]:
import requests
import json
from kafka import KafkaProducer
from kafka import KafkaConsumer

class IngestaoGenomic():

    def __init__(self, brokers):
        self.brokers = brokers
        self.kafka_producer = KafkaProducer(bootstrap_servers=brokers)

    def get_news_info(self,palavra_chave):
        url = f'https://newsapi.org/v2/everything?q={palavra_chave}'
        response = requests.get(url, headers={"X-Api-Key": "528d65099fb24bf6a49142f9af54ed02"})
        return response.json()['articles']
    
    def pub_news(self, palavras_chave:list):
        for palavra in palavras_chave:
            news = self.get_news_info(palavra)
            for new in news:
                self.kafka_producer.send("genomic", json.dumps(new).encode("utf-8"))

        print("Envio do lote de produtos finalizado com sucesso")



In [0]:
pipeline = IngestaoGenomic(brokers=["localhost:9092"])

pipeline.pub_news(["dna","rna"])

Envio do lote de produtos finalizado com sucesso


In [0]:
url = f'https://newsapi.org/v2/everything?q=rna'
response = requests.get(url, headers={"X-Api-Key": "528d65099fb24bf6a49142f9af54ed02"})